# Code de simulation de consommation des lampadaires optimisé

#### __Structure du code__
```txt
play.py
Code
    graph.py
    graph_save.py
    simul.py
    __pycache__  
Donnees
    carte.json
    carte_data.json
    save.json
```
***
### `play.py`

In [ ]:
"""Permet de générer une "interface" dans le terminal pour simplifier l'utilisation de l'ensemble des fonctions qui permet d'effectuer la simulation
"""
from Code.graph import *
from Code.graph_comparaison import *
from Code.graph_save import *
import os

def F():
    print(" ___                               _\n| _ \ __ _  _ _  __ _  _ __   ___ | |_  _ _  __ _  __ _  ___\n|  _// _` || '_|/ _` || '  \ / -_)|  _|| '_|/ _` |/ _` |/ -_)\n|_|  \__,_||_|  \__,_||_|_|_|\___| \__||_|  \__,_|\__, |\___|\n                                                  |___/")

def lecture(a:str)->tuple:
    """Convertie un str `A/B` en `(a,b)`

    Parametres
    ----------
    a : str
        le str sous forme `a/b`

    Renvoies
    -------
    tuple
        `(int(a), int(b))`
    """
    rep1 = ""
    rep2 = ""
    b = False
    for i in a:
        if i != "/" and b == False :
            rep1 += i
        elif i != "/" and b == True :
            rep2 += i
        elif i == "/":
            b = True
    return (int(rep1), int(rep2))
    
print(" ___  _               _        _    _                 _           _             _                                   _           _____  ___  ___  ___\n/ __|(_) _ __   _  _ | | __ _ | |_ (_) ___  _ _    __| | ___   __| | ___  _ __ | | __ _  __  ___  _ __   ___  _ _  | |_   ___  |_   _||_ _|| _ \| __|\n\__ \| || '  \ | || || |/ _` ||  _|| |/ _ \| ' \  / _` |/ -_) / _` |/ -_)| '_ \| |/ _` |/ _|/ -_)| '  \ / -_)| ' \ |  _| |___|   | |   | | |  _/| _|\n|___/|_||_|_|_| \_,_||_|\__,_| \__||_|\___/|_||_| \__,_|\___| \__,_|\___|| .__/|_|\__,_|\__|\___||_|_|_|\___||_||_| \__|         |_|  |___||_|  |___|\n                                                                         |_|")
graphe_type = int(input("Quel type de graphique vous voulez : \n 1. Normale \n 2. À partir d'une sauvegarde \n>>> "))

os.system('cls' if os.name == 'nt' else 'clear')
F()
if graphe_type == 1 :
    puissance = int(input("Puissance (en W) = "))
    if puissance == 0 :
        raise ValueError("La puissance ne peut pas être égale à 0")
    cst_tps = int(input("Constante de temps (en s) = "))
    iteration = int(input("Nombre de simulations ="))
    if iteration < 2 :
        raise ValueError("Le nombre de simulations est bien trop bas (min = 2)")
    tps_simulation = int(input("Temps de la simulation (en h) = "))
    if tps_simulation == 0 :
        raise ValueError("Le temps de simulation ne peut pas être égal à 0")
    nbr_utilisateur = int(input("Nombre d'utilisateurs ="))
    os.system('cls' if os.name == 'nt' else 'clear')
    F()
    arret = input("Activer la fonction d'arrêt des utilisateurs devant les lampadaires ? (o|n)\n>>>")
    if arret == "o" or arret == "oui" :
        os.system('cls' if os.name == 'nt' else 'clear')
        F()
        proba = input("Probabilité d'arrêt de l'utilisateur (ex : 1/10) =")
        A = lecture(proba)
        proba = [A[0], A[1], True]
    else :
        proba = [1, 10, False]
    os.system('cls' if os.name == 'nt' else 'clear')
    F()
    save = input("Activer la sauvegarde ? (o|n) \n>>> ")
    if save == "o" or save == "oui" :
        save = True
    else :
        save = False
    os.system('cls' if os.name == 'nt' else 'clear')
    graphe1(iteration, tps_simulation, cst_tps, puissance, nbr_utilisateur, proba, save)

elif graphe_type == 2:
    filepath = input("Chemin vers le fichier de sauvegarde (par défaut ='./Donnees/save.json') : \n>>> ")
    os.system('cls' if os.name == 'nt' else 'clear')
    graph_save(filepath)

***
## `Code`
***
### `graph.py`

In [ ]:
"""Permet de générer un graphique à partir d'une simulation
"""
from Code.simul import *
from matplotlib import pyplot
import numpy as np
import os

def graphe1(nbr_simulation:int, tps_simulation:int, cst_tps:int, puissance:int, nbr_utilisateur:int, prob:list = [1, 10, True], save:bool = False)->None:
    """Permet de simuler la consommation des lampadaires et de générer un graphique

    Parametres
    ----------
    nbr_simulation : int
        le nombre de simulations à effectuer
    tps_simulation : int
        le temps de la simulation
    cst_tps : int
        le temps d'allumage des lampadaires 
    puissance : int
        puissance des lamapdaires
    nbr_utilisateur : int
        nombre d'utilisateurs
    prob : list
        probabilité que l'utilisateur fasse un arrêt (devant un lampadaire) ou on aura [ numérateur, dénominateur, activé ou non], par défaut [1, 10, True]
    save : bool
        si on sauvegarde ou non les données ?, par défaut non
    Renvoies
    -------
    None
        effectue une simulation et génère un graphique
    """
    
    val = simulation(nbr_simulation, tps_simulation, cst_tps, puissance, nbr_utilisateur, prob, save)
    val_opti = [ i[0] for i in val["sim"] ]

    born_min = min(val_opti) - 5
    born_max = max(val_opti) + 5

    # méthode montecarlo
    uopti = (1/(nbr_simulation-1)*sum((np.array(val_opti)-val["moy"][0])**2.))**0.5
    
    os.system('cls' if os.name == 'nt' else 'clear')
    print(" ___                 _  _          _\n| _ \ ___  ___ _  _ | || |_  __ _ | |_  ___\n|   // -_)(_-<| || || ||  _|/ _` ||  _|(_-<\n|_|_\\\___|/__/ \_,_||_| \__|\__,_| \__|/__/\n")

    print("Valeur moyenne : \n")
    print(" - Optimisée : " + str(val["moy"][0]) + "wh")
    print(" - Normale : " + str(val["moy"][1]) + "wh \n")
    print("u(consomation optimisée) = " + str(uopti) + "\n")
    print("Ecart : " + str(((abs(val["moy"][0] - val["moy"][1]) / val["moy"][1]) * 100)) + "%\n")
    print("Temps : \n")
    print(" - Temps total : " + time_adap(val["tps_tot"]) + "\n")
    print("Paramètres : \n - Nombre de simulations : " + str(nbr_simulation) + "\n - Nombre d'utilisateurs : " + str(nbr_utilisateur) + "\n - Temps de simulation : " + str(tps_simulation) + "h")
    print("\n")
    ##################################################################################################################
    pyplot.hist(val_opti, range = (born_min, born_max), bins = 200, color = 'blue', edgecolor = 'black')
    pyplot.xlabel('consommation (en wh)')
    pyplot.ylabel('effectif')
    pyplot.title('Pour ' + str(len(val_opti)) + ' iterations - Consommation optimisé')
    pyplot.title('Pour ' + str(len(val_opti)) + ' iterations - Consommation optimisé')
    pyplot.show()

***
### `graph_save.py`

In [ ]:
"""Permet à partir d'un fichier de sauvegarde `.json` de régénéré le résultat de celle-ci
"""
import json
from matplotlib import pyplot
import numpy as np
import os
from Code.simul import *

def graph_save(filepath:str = "./Donnees/save.json")->None:
    """Permet de générer un graphique d'une simulation à partir d'une sauvegarde

    Parametres
    ----------
    filepath : str, optional
        le chemin vers la sauvegarde, by défaut "./Donnees/save.json"
    
    Renvoies
    -------
    None
    """
    with open(filepath) as json_carte:
        save = json.load(json_carte)

    iteration = save["parametre"]["nbr_simulation"]
    val = save
    val_opti = [ i[0] for i in save["rep_simulation"]["sim"] ]
    val_norm = [ i[1] for i in save["rep_simulation"]["sim"] ]

    born_min = min(val_opti) - 5
    born_max = max(val_opti) + 5

    # méthode montecarlo
    uopti = (1/(iteration-1)*sum((np.array(val_opti)-val["rep_simulation"]["moy"][0])**2.))**0.5
    
    os.system('cls' if os.name == 'nt' else 'clear')
    print(" ___                 _  _          _\n| _ \ ___  ___ _  _ | || |_  __ _ | |_  ___\n|   // -_)(_-<| || || ||  _|/ _` ||  _|(_-<\n|_|_\\\___|/__/ \_,_||_| \__|\__,_| \__|/__/\n")

    print("Valeur moyenne : \n")
    print(" - Optimisé : " + str(val["rep_simulation"]["moy"][0]) + "wh")
    print(" - Normale : " + str(val["rep_simulation"]["moy"][1]) + "wh \n")
    print("u(consommation optimisé) = " + str(uopti) + "\n")
    print("Ecart : " + str(((abs(val["rep_simulation"]["moy"][0] - val["rep_simulation"]["moy"][1]) / val["rep_simulation"]["moy"][1]) * 100)) + "%\n")
    print("Temps : \n")
    print(" - Temps total : " + str(time_adap(val["rep_simulation"]["tps_tot"])) + "\n")
    print("Paramètres : \n - Nombre de simulations : " + str(val["parametre"]["nbr_simulation"]) + "\n - Nombre d'utilisateurs : " + str(val["parametre"]["nbr_utilisateur"]) + "\n - Temps de simulation : " + str(val["parametre"]["tps_simulation"]) + "h" )
    print("\n")
    ##################################################################################################################
    pyplot.hist(val_opti, range = (born_min, born_max), bins = 200, color = 'blue', edgecolor = 'black')
    pyplot.xlabel('consommation (en wh)')
    pyplot.ylabel('effectif')
    pyplot.title('Pour ' + str(len(val_opti)) + ' iterations - Consommation optimisé')
    pyplot.show()

***
### `simul.py`

In [ ]:
"""Permet de simuler le déplacement aléatoire d'utilisateurs au sein d'une `carte' afin de déterminer la consommation de ces utilisateurs en fonction de leur déplacement (allumage de lampadaire)
"""
import json
from time import perf_counter
from random import * 
import sys
from math import *

with open('./Donnees/carte.json') as json_carte:
    carte = json.load(json_carte)
with open('./Donnees/carte_data.json') as json_carte:
    carte_data = json.load(json_carte)
    
Data = { i : {"nb_allumage" : 0, "tps_allumage" : 0} for i in range(len(carte)) } # initialisation de la liste des données
start = [ i for i in carte if carte[i]["entree/sortie"] == True ]
dist_tot = carte_data["dist_tot"] #formule : 20m * (nb_de_distance = len(carte) + nb_d_intersection - le_nb_ligne)
dim_voiture_moyenne = 4.7 #m

lien_sens = {
    "N" : ["NO", "NE"],
    "S" : ["SO", "SE"],
    "O" : ["NO", "SO"],
    "E" : ["NE", "SE"],
    "NO" : ["O", "N"],
    "NE" : ["N", "E"],
    "SO" : ["S", "O"],
    "SE" : ["S", "E"]
}
ecart_lampadaire = 20 #en m

chg = " ___  _               _        _    _\n/ __|(_) _ __   _  _ | | __ _ | |_ (_) ___  _ _\n\__ \| || '  \ | || || |/ _` ||  _|| |/ _ \| ' \  _  _  _\n|___/|_||_|_|_| \_,_||_|\__,_| \__||_|\___/|_||_|(_)(_)(_)\n"

# on vérifie qu'il n'y a pas d'erreur dans le fichier
err = []
for i in carte :
    for p in ["N", "S", "O", "E", "NO", "NE", "SO", "SE"] :
        for s in carte[i][p] : 
            R = s in carte[i]["voisins"] + [0]
            if R != True :
                err.append(i)
if len(err) != 0 :
    val_err = "|"
    for i in err :
        val_err += " " + i + " |"
    raise SyntaxError("Les données des points suivants sont mal éditées : " + val_err)
# on vérifie qu'il n'y a pas d'erreur dans le fichier

def time_adap(a:int)->str:
    """Permet de convertir un temps en second en fonction de sa valeur

    Parametres
    ----------
    a : int
        le temps en seconde

    Renvoies
    --------
    str
        le temps converti
    """
    if a >= 86400 : #jours
        return str(round(a/86400)) + "j"
    if a >= 3600 : #heure
        return str(round(a/3600)) + "h"
    if a >= 60 : #min
        return str(round(a/60)) + "min"
    if a < 60 : #second
        return str(a) + "s"

def updt(total, progress, prefix:str = "Calcul en cours : ", dim:int = 40)->sys:
    """
    Displays or updates a console progress bar.

    Original source: https://stackoverflow.com/a/15860757/1391441
    """
    etat = progress
    fin = total
    barLength, status = dim, ""
    progress = float(progress) / float(total)
    if progress >= 1.:
        progress, status = 1, "\r\n"
    block = int(round(barLength * progress))
    text = "\r" + prefix + str(etat) + "/" + str(fin) + " " + "[{}] {:.0f}% {}".format(
        "#" * block + "." * (barLength - block), round(progress * 100, 0),
        status)
    sys.stdout.write(text)
    sys.stdout.flush()

def cal_vit_tps(intervale:list = [4, 130])->tuple:
    """Permet de générer une vitesse aléatoire suivant un intervalle et de l'associer à un temps d'allumage du lampadaire

    Parametres
    ----------
    intervale : list, optionnel 
        l'intervalle suivant lequel on prendra les vitesses (en km/h), par défaut [4, 130]

    Renvoies
    --------
    tuple
        un tuple de la vitesse et du temps associé : `(vit, tps)`
    """
    vit = randint(intervale[0], intervale[1]) #en km/h
    tps = ceil(ecart_lampadaire / (vit/3.6)) # en s | on arrondit au supérieur, car on veut une valeur rond max pour que le lampadaire soit allumé le plus longtemps possible
    return (vit, tps)

def max_user(u:int)->True:
    """Permet de déterminer le nombre max d'utilisateurs possible dans la carte donnée

    Parametres
    ----------
    u : int
        le nombre d'utilisateurs de la simulation

    Renvoies
    --------
    True
        Si tout se passe bien 

    Exceptions
    ----------
    ValueError
        Dans le cas où la valeur dépasse le nombre d'utilisateurs max de la carte, on crée une erreur
    """
    if round(dist_tot/dim_voiture_moyenne) < u :
        raise ValueError("Vous avez dépassé le nombre max d'utilisateurs possible sur cette carte : " + str(u) + " > " + str(round(dist_tot/dim_voiture_moyenne)))
    return True

def temps_allumage(vitesse:float, temps:float)->float :
    """Permet de calculer le temps d'allumage des lampadaires en fonction de la vitesse des voitures en prenant en compte l'avance d'allumage des lampadaires.
On prend en référence : 10m/s -> 2 lampadaires d'allumer donc 1 d'avance (où on a 10m = ecart_lampadaire / 2 pour l'exemple) ont fait un produit en croix pour avoir les valeurs
    Parametres
    ----------
    vitesse : float
        la vitesse de l'utilisateur en km/h
    temps : float
        le temps d'allumage du lampadaire principal

    Renvoies
    --------
    float
        le temps total d'allumage des lampadaires
    """
    vit_ms = vitesse / 3.6 # km/h -> m/s
    nb_lampe_avance = ceil((vit_ms * 2) / (ecart_lampadaire / 2)) # nombre de lampadaires d'avance
    tps_tot = temps
    for i in range(2, nb_lampe_avance + 1) : # on calcule le temps que les lampadaires supplémentaires vont rester allumés
        tps_tot += temps / i 
    return tps_tot

def trajet(tps_simulation:int, vitesse:float, prob:list = [1, 20, True], int:tuple = (1, 5))->list:
    """Permet de simuler le déplacement d'un utilisateur suivant une direction
    
    Parametres
    ----------
    tps_simulation : int
        temps de la simulation
    vitesse : float
        vitesse de l'utilisateur 
    prob : list, optionnel 
        probabilité que l'utilisateur s'arrête devant un lampadaire (1/10 => [1, 10, "si on active ou pas : bool"]), par défaut [1, 20, True]
    int : tuple, optionnel 
        le nombre de tentatives pour réaliser la probabilité ("de" , " a "), par défaut (1, 5)

    Renvoies
    --------
    list
        renvoie la liste des lampadaires à allumer
    """

    def next(A:str)->str:
        """Permet de donner le prochain lampadaire qui sera allumé

        Parametres
        ----------
        A : str
            le lampadaire où on se trouve

        Renvoies
        -------
        str
            le prochain lampadaire
        """
        N = carte[A][orientation][randint(0, len(carte[A][orientation]) - 1)]
        if N == 0 :
            R = lien_sens[orientation][:]
            shuffle(R)
            for i in R :
                N = carte[A][i][randint(0, len(carte[A][i]) - 1)]
                if N != 0 :
                    return str(N)
            if N == 0 : 
                B = [ p for i in ["N", "S", "O", "E", "NO", "NE", "SO", "SE"] for p in carte[A][i] if p != 0]
                N = B[randint(0, len(B) - 1)]
                return str(N)
        else :
            return str(N)
    
    sens = ["N", "S", "O", "E", "NO", "NE", "SO", "SE"]
    orientation = sens[randint(0, len(start) - 1)]
    begin = start[randint(0, len(start) - 1)]
    trajet = [begin]
    begin = next(begin)
    trajet.append(begin)
    
    trajet_tot = []
    distance_max = vitesse * tps_simulation # on calcule la distance max que peuvent parcourir les utilisateurs en fonction de leur temps imparti
    lampadaire_max = round(distance_max / 0.02)
    while lampadaire_max - len(trajet_tot) > 0 :
        while carte[begin]["entree/sortie"] == False :
            mem = begin
            i = 0
            while begin in trajet :
                begin = next(mem)
                i += 1
                if i >= 10 :
                    break
            trajet.append(begin)
            if prob[2] == True :
                for _ in range(randint(int[0], int[1])) :
                    arret = randint(1, prob[1])
                    if arret <= prob[0]:
                        trajet.append(begin)
        trajet_tot += trajet
        begin = start[randint(0, len(start) - 1)]
        trajet = [begin]
        begin = next(begin)
        trajet.append(begin)
        orientation = sens[randint(0, len(start) - 1)]
    return trajet_tot
 
def adaptation(trajet:list, vitesse:float, tps_simulation:int)->list:
    """Permet de lier la vitesse de l'utilisateur à son déplacement ainsi qu'à sa vitesse, on considère une liste ou chaque élément représentant 0.25s

    Parametres
    ----------
    trajet : list
        la liste du trajet de l'utilisateur
    vitesse : float
        la vitesse de l'utilisateur
    tps_simulation : int
        le temps de la simulation 

    Renvoies
    --------
    list
        liste avec les nouveaux paramètres pris en compte

    Exceptions
    ----------
    ValueError
       3.6 < vitesse < 200 (en km/h), pour pouvoir réaliser le calcul
    """
    if vitesse < 3.6 :
        raise ValueError("La vitesse est bien trop petite pour réaliser le calcul (vitesse_min = 3.6km/h)")
    if vitesse > 200 :
        raise ValueError("La vitesse est bien trop élevée pour réaliser le calcul (vitesse_max = 200km/h, la résolution des calculs limite la vitesse max)")
    A = [ 0 for _ in range(round((0.25 * tps_simulation) / 6.94444e-5)) ] # pas de 0.25 pour le niveau de précision, on a donc une vitesse max de 130km/h | Pour la formule, il s'agit d'un produit en croix
    V = round(ecart_lampadaire/((vitesse / 3.6) * 0.25)) # on a ici le nombre de point à parcourir avant d'allumer un lampadaire | On passe la vitesse pour 0.25 | Pour la formule, il s'agit d'un produit en croix
    r = 0
    j = 0
    z = 0
    for i in range(len(A)) :
        r += 1
        if r == V :
            b = 0
            if j < len(trajet) :
                b = trajet[j]
                z = i
            A[i] += int(b)
            j += 1
            r = 0
    A = [ A[i] for i in range(len(A)) if i <= z ]
    
    # prise en compte de la taille des voitures
    B = A[:] + [ 0 for _ in range(V-1) ]
    taille_voiture = ceil(dim_voiture_moyenne/(ecart_lampadaire/V)) - 1 # en nombre de points | on retire 1, car on ne compte pas le point déjà mis
    if taille_voiture > 0 :
        place = []
        for i in range(len(A)) :
            if A[i] != 0 :
                place.append((i, A[i]))
        for i in place : 
            for p in range(1, taille_voiture) :
                B[i[0] + p] = i[1]
    # prise en compte de la taille des voitures
    return B
    
def deplacement(tps_simulation:int, nbr_utilisateur:int, prob:list = [1, 10, True])->dict:
    """Permet de simuler le déplacement simultané de plusieurs utilisateurs en même temps sur un temps donné pour un nombre donné d'utilisateurs

    Parametres
    ----------
    tps_simulation : int
        le temps de la simulation
    nbr_utilisateur : int
        le nombre d'utilisateurs
    prob : list
        probabilité que l'utilisateur fasse un arrêt (devant un lampadaire) ou on aura [ numérateur, dénominateur, activé ou non], par défaut [1, 10, True]

    Renvoies
    -------
    dict
        renvoie alors les trajets, les vitesses et le nombre de lampadaires allumable par les utilisateurs
    """
    Data_d = {}
    for i in range(nbr_utilisateur):
        vit_tps = cal_vit_tps()
        vitesse_utilisateur = vit_tps[0] # on choisit une vitesse aléatoire pour l'utilisateur
        trajet_utilisateur = adaptation(trajet(tps_simulation, vitesse_utilisateur, prob), vitesse_utilisateur, tps_simulation)
        distance_max = vitesse_utilisateur * tps_simulation # on calcule la distance max que peuvent parcourir les utilisateurs en fonction de leur temps imparti
        lampadaire_max = round(distance_max / 0.02) # on détermine le nombre max de lampadaires qu'ils peuvent allumer en fonction de leur vitesse et du temps de l'expérimentation | on a des lampadaires espacés de 20m = 0,02km
        if len(trajet_utilisateur) > lampadaire_max : # s'il y a trop de lampadaire allumé lors du trajet, on en retire
            trajetV2 = [ trajet_utilisateur[i] for i in range(lampadaire_max) ]
            trajet_utilisateur = trajetV2
        Data_d[i] = {
            "trajet" : trajet_utilisateur,
            "vitesse" : vitesse_utilisateur,
            "lampadaire_max" : lampadaire_max,
            "temps" : ceil(temps_allumage(vitesse_utilisateur, vit_tps[1])) # on arrondit au supérieur pour avoir une valeur ronde pour que le lampadaire reste le max possible de temps allumé
        }
    
    return Data_d

def fusion(data:dict)->dict:
    """Permet de rendre toutes les listes de la même taille pour faciliter la comparaison

    Parametres
    ----------
    data : dict
        les données de déplacement de l'utilisateur (vient de la fonction déplacement)

    Renvoies
    -------
    dict
        la liste des déplacements uniformisés
    """
    up = max([ len(data[i]["trajet"]) for i in data ]) # on prend l'utilisateur avec le plus de lampadaire d'allumer
    data_harmo = { i : (data[i]["trajet"]) + [ 0 for _ in range(up - len(data[i]["trajet"])) ] if (len(data[i]["trajet"]) < up) else data[i]["trajet"] for i in data }
    return data_harmo

def deplacement_affectation(data:dict, data_deplacement:dict)->list:
    """Permet de renvoyer la liste d'allumage des lampadaire en prenant en compte le fait qu'il peut y avoir plusieurs utilisateurs au même endroit au même moment

    Parametres
    ----------
    data : dict
        la liste qui vient de fusion et qui va permettre de mettre tout ensemble
    data_deplacement : dict 
        dictionnaire des données de déplacement des utilisateurs

    Renvoies
    -------
    list
        la liste avec le nombre d'allumages de chacun des utilisateurs
    """
    lampdaire_list = [ [] for _ in range(len(carte) + 1)] # on ne compte pas le 0 donc on ajoute 1 (la liste d'utilisateurs commence à 1 et se termine donc à n+1)
    calcule = [ lampdaire_list[int(data[p][i])].append(data_deplacement[p]["temps"]) for i in range(len(data[0])) for p in range(len(data)) if int(data[p][i]) != 0 and len(lampdaire_list[int(data[p][i])]) <= i]
    return lampdaire_list

def calcule(tps_simulation:int, puissance:int, cst_tps:int, data:list)->tuple:
    """Permet le calcul de la consommation

    Parametres
    ----------
    tps_simulation : int
        temps de la simulation
    puissance : int
        la puissance des lampadaires
    data : list
        les données du temps d'allumage des lampadaires

    Renvoies
    -------
    tuple
        (consommation optimisée, consommation classique)
    """
    simulation_classic = len(carte) # nombre de lampadaires
    tps_opti = 0 # le temps total lors de l'expérimentation optimisée
    for i in data :
        tps = 0
        for p in i :
            tps += p + cst_tps
        if tps > tps_simulation * 3600 :
            tps = tps_simulation * 3600
        tps_opti += tps 
    conso_opti = ((tps_opti)/3600) * puissance # calcul de la puissance
    conso_classic = (tps_simulation * simulation_classic) * puissance  
    return (round(conso_opti), round(conso_classic))

def f_save(data:dict, filepath = "./Donnees/save.json")->None:
    """Permet de sauvegarder les résultats de la simulation et ces paramètres dans un fichier

    Parametres
    ----------
    data : dict
        les données à sauvegarder
    filepath : path, optional
        le chemin vers le fichier à sauvegarder, by défaut "save.json"
    """
    with open(filepath, 'w') as mon_fichier: # on crée le fichier voulu et on l'enregistre à l'endroit souhaité
	    json.dump(data, mon_fichier)

def simulation(nbr_simulation:int, tps_simulation:int, cst_tps:int, puissance:int, nbr_utilisateur:int, proba:list = [1, 10, True], save:bool = False, info_sup:str = "")->dict:
    """Permet de simuler la consommation des lampadaires

    Parametres
    ----------
    nbr_simulation : int
        le nombre de simulations à effectuer
    tps_simulation : int
        le temps de la simulation 
    cst_tps : int
        le temps d'allumage des lampadaires 
    puissance : int
        puissance des lampadaires
    nbr_utilisateur : int
        nombre d'utilisateurs
    prob : list
        probabilité que l'utilisateur fasse un arrêt (devant un lampadaire) ou on aura [ numérateur, dénominateur, activé ou non], par défaut [1, 10, True]
    save : bool
        si on sauvegarde ou non les données ?, par défaut non
    info_sup : str
        Les infos supplémentaires sur la barre de chargement (/ ! ajouter " | " à la fin de votre info pour plus de lisibilité), par défaut ""

    Renvoies
    -------
    dict
        renvoie alors la consommation moyenne optimiser et celle, classique, ainsi que toutes les valeurs de simulation. ( { "sim" : [...], "moy" : (.., ..)} )
    """
    
    max_user(nbr_utilisateur)
       
    print(chg)
    simulation_L = []
    time_b = []
    updt(nbr_simulation, 0, info_sup + "Calcule : ")
    
    start = perf_counter()
    
    for i in range(nbr_simulation):
        start_in = perf_counter() # temps
        etape1 = deplacement(tps_simulation, nbr_utilisateur, proba)
        etape2 = fusion(etape1)
        etape3 = deplacement_affectation(etape2, etape1)
        etape4 = calcule(tps_simulation, puissance, cst_tps, etape3)
        simulation_L.append(etape4)
        end_in = perf_counter() # temps
        time_b.append(end_in - start_in) # temps calculé
        time_estim = round((sum(time_b)/len(time_b)) * (nbr_simulation - i))
        updt(nbr_simulation, i + 1, info_sup +  "Temps restant : " + time_adap(time_estim) + " | Calcule : ")
    
    end = perf_counter()
    
    print()
    optimiser_list = [ i[0] for i in simulation_L ] # on fait les moyennes
    classic_list = [ i[1] for i in simulation_L ]
    moy_opti = sum(optimiser_list)/len(simulation_L)
    moy_classic = sum(classic_list)/len(simulation_L)
    
    rep = {
        "sim" : simulation_L,
        "moy" : (moy_opti, moy_classic),
        "tps_tot" : end - start,
    }
    
    if save == True :
        data = {"rep_simulation" : rep, "parametre" : {"nbr_simulation" : nbr_simulation, "tps_simulation" : tps_simulation, "cst_tps" : cst_tps, "puissance": puissance, "nbr_utilisateur": nbr_utilisateur, "prob" : proba,  "save": save}}
        f_save(data) 
    return rep

***
## `Donnees`
***
### `carte.json`

In [ ]:
{
    "1": {
        "entree/sortie" : true,
        "voisins" : [0,2],
        "N" : [0],
        "S" : [0],
        "O" : [0],
        "E" : [2],
        "NO" : [0],
        "NE" : [0], 
        "SO" : [0],
        "SE" : [0]
    },
    "2": {
        "entree/sortie" : false,
        "voisins" : [1,3],
        "N" : [0],
        "S" : [0],
        "O" : [1],
        "E" : [3],
        "NO" : [0],
        "NE" : [0], 
        "SO" : [0],
        "SE" : [0]
    },
    "3": {
        "entree/sortie" : false,
        "voisins" : [2,4],
        "N" : [0],
        "S" : [0],
        "O" : [2],
        "E" : [4],
        "NO" : [0],
        "NE" : [0], 
        "SO" : [0],
        "SE" : [0]
    },
    "4": {
        "entree/sortie" : false,
        "voisins" : [3,5],
        "N" : [0],
        "S" : [0],
        "O" : [3],
        "E" : [5],
        "NO" : [0],
        "NE" : [0], 
        "SO" : [0],
        "SE" : [0]
    },
    "5": {
        "entree/sortie" : false,
        "voisins" : [4,6],
        "N" : [0],
        "S" : [0],
        "O" : [4],
        "E" : [6],
        "NO" : [0],
        "NE" : [0], 
        "SO" : [0],
        "SE" : [0]
    },
    "6": {
        "entree/sortie" : false,
        "voisins" : [5,7,31,32],
        "N" : [31],
        "S" : [32],
        "O" : [5],
        "E" : [7],
        "NO" : [5, 31],
        "NE" : [7, 31], 
        "SO" : [5, 32],
        "SE" : [7, 32]
    },
    "7": {
        "entree/sortie" : false,
        "voisins" : [6,8],
        "N" : [0],
        "S" : [0],
        "O" : [6],
        "E" : [8],
        "NO" : [0],
        "NE" : [0], 
        "SO" : [0],
        "SE" : [0]
    },
    "8": {
        "entree/sortie" : false,
        "voisins" : [7,9],
        "N" : [0],
        "S" : [0],
        "O" : [7],
        "E" : [9],
        "NO" : [0],
        "NE" : [0], 
        "SO" : [0],
        "SE" : [0]
    },
    "9": {
        "entree/sortie" : false,
        "voisins" : [8,10,44,45],
        "N" : [44],
        "S" : [45],
        "O" : [8],
        "E" : [10],
        "NO" : [8, 44],
        "NE" : [10, 44], 
        "SO" : [8, 45],
        "SE" : [10, 45]
    },
    "10": {
        "entree/sortie" : false,
        "voisins" : [9,11],
        "N" : [0],
        "S" : [0],
        "O" : [9],
        "E" : [11],
        "NO" : [0],
        "NE" : [0], 
        "SO" : [0],
        "SE" : [0]
    },
    "11": {
        "entree/sortie" : false,
        "voisins" : [10,12,48],
        "N" : [0],
        "S" : [48],
        "O" : [10],
        "E" : [12],
        "NO" : [10],
        "NE" : [12], 
        "SO" : [10, 48],
        "SE" : [12, 48]
    },
    "12": {
        "entree/sortie" : false,
        "voisins" : [11,13],
        "N" : [0],
        "S" : [0],
        "O" : [11],
        "E" : [13],
        "NO" : [0],
        "NE" : [0], 
        "SO" : [0],
        "SE" : [0]
    },
    "13": {
        "entree/sortie" : true,
        "voisins" : [12,0],
        "N" : [0],
        "S" : [0],
        "O" : [12],
        "E" : [0],
        "NO" : [0],
        "NE" : [0], 
        "SO" : [0],
        "SE" : [0]
    },
    "14": {
        "entree/sortie" : true,
        "voisins" : [0,15],
        "N" : [0],
        "S" : [0],
        "O" : [0],
        "E" : [15],
        "NO" : [0],
        "NE" : [0], 
        "SO" : [0],
        "SE" : [0]
    },
    "15": {
        "entree/sortie" : false,
        "voisins" : [14,16],
        "N" : [0],
        "S" : [0],
        "O" : [14],
        "E" : [16],
        "NO" : [0],
        "NE" : [0], 
        "SO" : [0],
        "SE" : [0]
    },
    "16": {
        "entree/sortie" : false,
        "voisins" : [15,17],
        "N" : [0],
        "S" : [0],
        "O" : [15],
        "E" : [17],
        "NO" : [0],
        "NE" : [0], 
        "SO" : [0],
        "SE" : [0]
    },
    "17": {
        "entree/sortie" : false,
        "voisins" : [16,18],
        "N" : [0],
        "S" : [0],
        "O" : [16],
        "E" : [18],
        "NO" : [0],
        "NE" : [0], 
        "SO" : [0],
        "SE" : [0]
    },
    "18": {
        "entree/sortie" : false,
        "voisins" : [17,19],
        "N" : [0],
        "S" : [0],
        "O" : [17],
        "E" : [19],
        "NO" : [0],
        "NE" : [0], 
        "SO" : [0],
        "SE" : [0]
    },
    "19": {
        "entree/sortie" : false,
        "voisins" : [18,20,34,35],
        "N" : [34],
        "S" : [35],
        "O" : [18],
        "E" : [20],
        "NO" : [18, 34],
        "NE" : [20, 34], 
        "SO" : [18, 35],
        "SE" : [20, 35]
    },
    "20": {
        "entree/sortie" : false,
        "voisins" : [19,21],
        "N" : [0],
        "S" : [0],
        "O" : [19],
        "E" : [21],
        "NO" : [0],
        "NE" : [0], 
        "SO" : [0],
        "SE" : [0]
    },
    "21": {
        "entree/sortie" : false,
        "voisins" : [20,22],
        "N" : [0],
        "S" : [0],
        "O" : [20],
        "E" : [22],
        "NO" : [0],
        "NE" : [0], 
        "SO" : [0],
        "SE" : [0]
    },
    "22": {
        "entree/sortie" : false,
        "voisins" : [21,23,47],
        "N" : [47],
        "S" : [0],
        "O" : [21],
        "E" : [23],
        "NO" : [21, 47],
        "NE" : [23, 47], 
        "SO" : [21],
        "SE" : [23]
    },
    "23": {
        "entree/sortie" : false,
        "voisins" : [22,24],
        "N" : [0],
        "S" : [0],
        "O" : [22],
        "E" : [24],
        "NO" : [0],
        "NE" : [0], 
        "SO" : [0],
        "SE" : [0]
    },
    "24": {
        "entree/sortie" : false,
        "voisins" : [23,25,50,51],
        "N" : [50],
        "S" : [51],
        "O" : [23],
        "E" : [25],
        "NO" : [23, 50],
        "NE" : [25, 50], 
        "SO" : [23, 51],
        "SE" : [25, 51]
    },
    "25": {
        "entree/sortie" : false,
        "voisins" : [24,26],
        "N" : [0],
        "S" : [0],
        "O" : [24],
        "E" : [26],
        "NO" : [0],
        "NE" : [0], 
        "SO" : [0],
        "SE" : [0]
    },
    "26": {
        "entree/sortie" : true,
        "voisins" : [25,0],
        "N" : [0],
        "S" : [0],
        "O" : [25],
        "E" : [0],
        "NO" : [0],
        "NE" : [0], 
        "SO" : [0],
        "SE" : [0]
    },
    "27": {
        "entree/sortie" : true,
        "voisins" : [0,28],
        "N" : [0],
        "S" : [28],
        "O" : [0],
        "E" : [0],
        "NO" : [0],
        "NE" : [0], 
        "SO" : [0],
        "SE" : [0]
    },
    "28": {
        "entree/sortie" : false,
        "voisins" : [27,29],
        "N" : [27],
        "S" : [29],
        "O" : [0],
        "E" : [0],
        "NO" : [0],
        "NE" : [0], 
        "SO" : [0],
        "SE" : [0]
    },
    "29": {
        "entree/sortie" : false,
        "voisins" : [28,30],
        "N" : [28],
        "S" : [30],
        "O" : [0],
        "E" : [0],
        "NO" : [0],
        "NE" : [0], 
        "SO" : [0],
        "SE" : [0]
    },
    "30": {
        "entree/sortie" : false,
        "voisins" : [29,31],
        "N" : [29],
        "S" : [31],
        "O" : [0],
        "E" : [0],
        "NO" : [0],
        "NE" : [0], 
        "SO" : [0],
        "SE" : [0]
    },
    "31": {
        "entree/sortie" : false,
        "voisins" : [30,6],
        "N" : [30],
        "S" : [6],
        "O" : [0],
        "E" : [0],
        "NO" : [0],
        "NE" : [0], 
        "SO" : [0],
        "SE" : [0]
    },
    "32": {
        "entree/sortie" : false,
        "voisins" : [6,33],
        "N" : [6],
        "S" : [33],
        "O" : [0],
        "E" : [0],
        "NO" : [0],
        "NE" : [0], 
        "SO" : [0],
        "SE" : [0]
    },
    "33": {
        "entree/sortie" : false,
        "voisins" : [32,34],
        "N" : [32],
        "S" : [34],
        "O" : [0],
        "E" : [0],
        "NO" : [0],
        "NE" : [0], 
        "SO" : [0],
        "SE" : [0]
    },
    "34": {
        "entree/sortie" : false,
        "voisins" : [33,19],
        "N" : [33],
        "S" : [19],
        "O" : [0],
        "E" : [0],
        "NO" : [0],
        "NE" : [0], 
        "SO" : [0],
        "SE" : [0]
    },
    "35": {
        "entree/sortie" : false,
        "voisins" : [19,36],
        "N" : [19],
        "S" : [36],
        "O" : [0],
        "E" : [0],
        "NO" : [0],
        "NE" : [0], 
        "SO" : [0],
        "SE" : [0]
    },
    "36": {
        "entree/sortie" : false,
        "voisins" : [35,37],
        "N" : [35],
        "S" : [37],
        "O" : [0],
        "E" : [0],
        "NO" : [0],
        "NE" : [0], 
        "SO" : [0],
        "SE" : [0]
    },
    "37": {
        "entree/sortie" : false,
        "voisins" : [36,38],
        "N" : [36],
        "S" : [38],
        "O" : [0],
        "E" : [0],
        "NO" : [0],
        "NE" : [0], 
        "SO" : [0],
        "SE" : [0]
    },
    "38": {
        "entree/sortie" : true,
        "voisins" : [37,0],
        "N" : [37],
        "S" : [0],
        "O" : [0],
        "E" : [0],
        "NO" : [0],
        "NE" : [0], 
        "SO" : [0],
        "SE" : [0]
    },
    "39": {
        "entree/sortie" : true,
        "voisins" : [0,40],
        "N" : [0],
        "S" : [40],
        "O" : [0],
        "E" : [0],
        "NO" : [0],
        "NE" : [0], 
        "SO" : [0],
        "SE" : [0]
    },
    "40": {
        "entree/sortie" : false,
        "voisins" : [39,41],
        "N" : [39],
        "S" : [41],
        "O" : [0],
        "E" : [0],
        "NO" : [0],
        "NE" : [0], 
        "SO" : [0],
        "SE" : [0]
    },
    "41": {
        "entree/sortie" : false,
        "voisins" : [40,42],
        "N" : [40],
        "S" : [42],
        "O" : [0],
        "E" : [0],
        "NO" : [0],
        "NE" : [0], 
        "SO" : [0],
        "SE" : [0]
    },
    "42": {
        "entree/sortie" : false,
        "voisins" : [41,43],
        "N" : [41],
        "S" : [43],
        "O" : [0],
        "E" : [0],
        "NO" : [0],
        "NE" : [0], 
        "SO" : [0],
        "SE" : [0]
    },
    "43": {
        "entree/sortie" : false,
        "voisins" : [42,44],
        "N" : [42],
        "S" : [44],
        "O" : [0],
        "E" : [0],
        "NO" : [0],
        "NE" : [0], 
        "SO" : [0],
        "SE" : [0]
    },
    "44": {
        "entree/sortie" : false,
        "voisins" : [43,9],
        "N" : [43],
        "S" : [9],
        "O" : [0],
        "E" : [0],
        "NO" : [0],
        "NE" : [0], 
        "SO" : [0],
        "SE" : [0]
    },
    "45": {
        "entree/sortie" : false,
        "voisins" : [9,46],
        "N" : [9],
        "S" : [46],
        "O" : [0],
        "E" : [0],
        "NO" : [0],
        "NE" : [0], 
        "SO" : [0],
        "SE" : [0]
    },
    "46": {
        "entree/sortie" : false,
        "voisins" : [45,47],
        "N" : [45],
        "S" : [47],
        "O" : [0],
        "E" : [0],
        "NO" : [0],
        "NE" : [0], 
        "SO" : [0],
        "SE" : [0]
    },
    "47": {
        "entree/sortie" : false,
        "voisins" : [46,22],
        "N" : [46],
        "S" : [22],
        "O" : [0],
        "E" : [0],
        "NO" : [0],
        "NE" : [0], 
        "SO" : [0],
        "SE" : [0]
    },
    "48": {
        "entree/sortie" : false,
        "voisins" : [11,49],
        "N" : [11],
        "S" : [49],
        "O" : [0],
        "E" : [0],
        "NO" : [0],
        "NE" : [0], 
        "SO" : [0],
        "SE" : [0]
    },
    "49": {
        "entree/sortie" : false,
        "voisins" : [48,50],
        "N" : [48],
        "S" : [50],
        "O" : [0],
        "E" : [0],
        "NO" : [0],
        "NE" : [0], 
        "SO" : [0],
        "SE" : [0]
    },
    "50": {
        "entree/sortie" : false,
        "voisins" : [49,24],
        "N" : [49],
        "S" : [24],
        "O" : [0],
        "E" : [0],
        "NO" : [0],
        "NE" : [0], 
        "SO" : [0],
        "SE" : [0]
    },
    "51": {
        "entree/sortie" : false,
        "voisins" : [24,52],
        "N" : [24],
        "S" : [52],
        "O" : [0],
        "E" : [0],
        "NO" : [0],
        "NE" : [0], 
        "SO" : [0],
        "SE" : [0]
    },
    "52": {
        "entree/sortie" : false,
        "voisins" : [51,53],
        "N" : [51],
        "S" : [53],
        "O" : [0],
        "E" : [0],
        "NO" : [0],
        "NE" : [0], 
        "SO" : [0],
        "SE" : [0]
    },
    "53": {
        "entree/sortie" : true,
        "voisins" : [52,0],
        "N" : [52],
        "S" : [0],
        "O" : [0],
        "E" : [0],
        "NO" : [0],
        "NE" : [0], 
        "SO" : [0],
        "SE" : [0]
    }
}

***
### `carte_data.json`

In [ ]:
{
    "dist_tot" : 1080
}

***
### `save.json`

In [ ]:
{
    "rep_simulation": {
        "sim": [
            [
                130,
                3710
            ],
            [
                146,
                3710
            ],
            [
                193,
                3710
            ],
            [
                156,
                3710
            ],
            [
                141,
                3710
            ],
            [
                158,
                3710
            ],
            [
                175,
                3710
            ],
            [
                178,
                3710
            ],
            [
                176,
                3710
            ],
            [
                176,
                3710
            ],
            [
                107,
                3710
            ],
            [
                154,
                3710
            ],
            [
                129,
                3710
            ],
            [
                141,
                3710
            ],
            [
                124,
                3710
            ],
            [
                149,
                3710
            ],
            [
                151,
                3710
            ],
            [
                91,
                3710
            ],
            [
                111,
                3710
            ],
            [
                130,
                3710
            ]
        ],
        "moy": [
            145.8,
            3710.0
        ],
        "tps_tot": 1.3091330528259277
    },
    "parametre": {
        "nbr_simulation": 10,
        "tps_simulation": 1,
        "cst_tps": 0,
        "puissance": 70,
        "nbr_utilisateur": 10,
        "type": 1,
        "nbr_lampadaire": 0,
        "fonction": 1,
        "prob": [
            1,
            10,
            false
        ],
        "save": true
    }
}

# Code Montage expérimentale

```cpp
#include <Arduino.h>

int sensorValue; // valeur de sortie de la photorésistance 
int lux; // variable qui stocke la luminosité en LUX

int temps_allumer = 4000; // temps d'allumage (varie en fonction de la vitesse) 
int variation_distance1 = 0; // distance n°1 (t-1)
int variation_distance2 = 0; // distance n°2 (t)
int variation_rep = 0; 
int variation_val = 0;
int esp_number_val = 0; // nombre d'esp à allumer
int lumiere_val = 0; // valeur de l'intensité lumineuse 
bool reception_esp = false; // condition sur la réception de donnée

// variable reçue par l'esp
int incoming_esp_number = 0;
int incoming_temps = 0;
int incoming_lumiere = 0;
int incoming_start = 0;
// variable reçue par l'esp

float distanceCm; // distance donnée par le capteur ultrason

long duration;

const int esp_number = 1; // numéro de l'esp
const int const_temps = 1; // constante de temps 
const int cap_luminere = 3800; // (5 pour la réalité)
const int dist_mini_immobile = 250; // distance minimale en cm pour définir une personne immobile 
const int dist_max = 270; // distance maximale de détection (dont la limite pour éviter les erreurs du capteur)
const int temps_allume_immobile = 4000; // temps d'allumage pour les personnes immobiles
const int nombre_tot = 3; // nombre total de lampadaires

const int trigPin = 5;
const int echoPin = 18;

// bibliothèque pour la connexion réseau entre esp
#include "WiFi.h" 
#include <esp_now.h>
// bibliothèque pour la connexion réseau entre esp

#define SOUND_SPEED 0.034
#define LED 23

#define VIN 5 //tension du capteur de lumière
#define R 10000 // résistance du capteur de lumière 

// fonction 
int sensorRawToPhys(int raw){ // conversion de la valeur du capteur vers des lumen 
  float Vout = float(raw) * (VIN / float(1024));// conversion analogique vers un voltage 
  float RLDR = (R * (VIN - Vout))/Vout; // conversion du voltage vers la résistance 
  int phys=500/(RLDR/1000); // conversion de la résistance en lumen
  return phys;
}
// fonction

esp_now_peer_info_t peerInfo;
// adresse mac du récepteur
uint8_t broadcastAddress[2][6] = {
  {0xC8, 0xC9, 0xA3, 0xFA, 0xAA, 0x08},//station2
  {0x94, 0xB9, 0x7E, 0xE9, 0x0F, 0x04} //station3
  
};

// variable de stockage du succès de la transmission 
String success;

// message d'envoi
typedef struct struct_message {
    int esp_number;
    int temps;
    int lumiere;
    int start;
} struct_message;

// création d'une structure d'envoi
struct_message sendReading;

// création d'une structure de réception
struct_message incomingReadings;

// fonction d'envoi de données
void OnDataSent(const uint8_t *mac_addr, esp_now_send_status_t status) { // envoie des données 
  Serial.print("\r\nLast Packet Send Status:\t");
  Serial.println(status == ESP_NOW_SEND_SUCCESS ? "Delivery Success" : "Delivery Fail");
  if (status == 0){
    success = "Delivery Success";
  }
  else{
    success = "Delivery Fail";
  }
}

void OnDataRecv(const uint8_t * mac, const uint8_t *incomingData, int len) { // réception des données
  memcpy(&incomingReadings, incomingData, sizeof(incomingReadings));
  Serial.print("Bytes received: ");
  Serial.println(len);
  reception_esp = true; // on reçoit des données
  // passage des données reçues dans les variables du programme
  incoming_esp_number = incomingReadings.esp_number;
  incoming_temps = incomingReadings.temps;
  incoming_lumiere = incomingReadings.lumiere;
  incoming_start = incomingReadings.start;
  // passage des données reçues dans les variables du programme
}
// fonction d'envoi de données

void setup() // initialisation
{
  Serial.begin(9600); // vitesse de communication 

  pinMode(LED, OUTPUT); // initialisation de la led 

  // initialisation de du capteur ultrason
  pinMode(trigPin, OUTPUT);
  pinMode(echoPin, INPUT);
  // initialisation de du capteur ultrason

  WiFi.mode(WIFI_MODE_STA); // adresse mac de l'esp
  Serial.print("Mac Address : ");
  Serial.println(WiFi.macAddress());

  if (esp_now_init() != ESP_OK) {
  Serial.println("Error initializing ESP-NOW");
  }

  esp_now_register_send_cb(OnDataSent); // status des données envoyées
  
  //enregistrement de l’appairage
    memcpy(peerInfo.peer_addr, broadcastAddress, 6);
    peerInfo.channel = 0;  
    peerInfo.encrypt = false;
  // enregistrement de l’appairage
  
  // ajout d'un appairage     
  if (esp_now_add_peer(&peerInfo) != ESP_OK){
    Serial.println("Failed to add peer");
    return;
  }
  // ajout d'un appairage
     
  // Register for a callback function that will be called when data is received
  esp_now_register_recv_cb(OnDataRecv);
}
 
void loop() // programme
{
  if (reception_esp == true && incoming_esp_number >= esp_number){ // cas de réception des données 
    reception_esp = false; // on remet à false la variable de réception de données
    if (incoming_esp_number >= esp_number && incoming_start <= esp_number) { // vérification de si l'esp est concernée par l'appel
      digitalWrite(LED, incoming_lumiere); // modification de la lumière 
      delay(incoming_temps); // temps d'allumage 
    }
  } else { // cas de non réception de donnée
  // calcul de la distance
  digitalWrite(trigPin, LOW);
  delayMicroseconds(2);
  digitalWrite(trigPin, HIGH);
  delayMicroseconds(10);
  digitalWrite(trigPin, LOW);
  duration = pulseIn(echoPin, HIGH);
  distanceCm = duration * SOUND_SPEED/2;
  // calcul de la distance

  sensorValue = analogRead(A0); // valeur du capteur de lumière

  // variation de la distance
  variation_rep = variation_rep + 1; // on compte le nombre de valeurs enregistrées (max 2)
  if (variation_rep == 3) { // si 3, on remet à 1
    variation_rep = 1;
  }
  if (variation_rep == 1) { // on garde la distance à t-1
    variation_distance1 = distanceCm; 
  } else if (variation_rep == 2) { // on garde la distance à t
    variation_distance2 = distanceCm;
  }
  if (variation_distance1 > 0 && variation_distance2 > 0) { // calcul de la variation de la distance
    variation_val = abs(variation_distance2 - variation_distance1);
  } else {
    variation_val = 0; // dans le cas où on n'a pas encore les valeurs ≠ 0, on se met à 0
  }
  // interprétation
  if (variation_val == 0) {
    esp_number_val = 0;
  } else if (variation_val <= 2) { // <= 7,2km/h (pour une période de 10ms)
    esp_number_val = 1;
  } else if (variation_val <= 15) { // <= 54km/h (pour une période de 10ms)
    esp_number_val = 2;
  } else if (variation_val <= 20) { // <= 72km/h (pour une période de 10ms)
    esp_number_val = 3;
  } else if (variation_val <= 25) { // <= 90km/h (pour une période de 10ms)
    esp_number_val = 4;
  } else if (variation_val <= 30) { // <= 108km/h (pour une période de 10ms)
    esp_number_val = 5;
  } else if (variation_val > 30) {
    esp_number_val = 5;
  }
  // variation de la distance

  // temps d'allumage | Surface au sol supposer 28m^2 (diamètre = 6m) | Constance de temps * 4
  if (variation_val == 0) {
    temps_allumer = 0;
  } else if (variation_val <= 2) { // <= 7,2km/h -> 3s
    temps_allumer = 3000 * const_temps;
  } else if (variation_val <= 15) { // <= 54km/h -> 1,6s
    temps_allumer = 1600 * const_temps;
  } else if (variation_val <= 20) { // <= 72km/h -> 1,2s
    temps_allumer = 1200 * const_temps;
  } else if (variation_val <= 25) { // <= 90km/h -> 0,96s
    temps_allumer = 960 * const_temps;
  } else if (variation_val <= 30) { // <= 108km/h -> 0,8s
    temps_allumer = 800 * const_temps;
  } else if (variation_val > 30) {
    temps_allumer = 800 * const_temps;
  }
  // temps d'allumage

  // intensité de la led en fonction de la luminosité
  lumiere_val = ((1 + sensorValue)*255)/2100;
  if (sensorValue <= 5) {
    lumiere_val = 255;
  }
  // intensité de la led en fonction de la luminosité
  
  if (sensorValue + 1 < cap_luminere && (esp_number_val + esp_number - 1) >= esp_number && distanceCm <= dist_max) { // allumage en fonction de la vitesse de déplacement de l'utilisateur
    digitalWrite(LED, lumiere_val);
    variation_distance1 = 0;
    variation_distance2 = 0;

    // modification des variables d'envoi
    sendReading.esp_number = esp_number_val + esp_number - 1;
    sendReading.temps = temps_allumer;
    sendReading.lumiere = lumiere_val;
    sendReading.start = esp_number;
    // modification des variables d'envoi

   // envoie à l'aide d'ESP-NOW
   int i = nombre_tot;
   for (int i = 0; i < nombre_tot - 1; i++) {
      esp_err_t result = esp_now_send(broadcastAddress[i], (uint8_t *) &sendReading, sizeof(sendReading));
      if (result == ESP_OK) {
        Serial.print("Sent with success to : ");
        Serial.println(i);
      }
      else {
        Serial.print("Error sending the data to : ");
        Serial.println(i);
      }
   };

    delay(temps_allumer); // temps d'allumage 

  } else if (sensorValue + 1 < cap_luminere && esp_number_val == 0 && distanceCm <= dist_mini_immobile) { // utilisateur immobile sous le lampadaire
    digitalWrite(LED, lumiere_val);
    delay(temps_allume_immobile);
  } else { // pas d'utilisateur sous le lampadaire
    digitalWrite(LED, LOW); // on éteint la led
  }  
  delay(10); // 10ms
}
}
```